In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam

from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
# Import your model and data preparation modules
from models.Tals_Flame_AI_Model import Tals_Flame_AI_Model
from utils.data_preparation import load_and_prepare_data


import cv2
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib

In [3]:
import torch
import cv2
import numpy as np
from models.Tals_Flame_AI_Model import Tals_Flame_AI_Model

# Initialize the model
input_size = 21  # Update according to your model's input size
model = Tals_Flame_AI_Model(input_size=input_size)
model_save_path = 'path_to_save_model_weights.pth'

# Load the model weights
model.load_state_dict(torch.load(model_save_path))
model.eval()  # Set the model to evaluation mode

def extract_features_from_frame(frame):
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply threshold to get binary image
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Placeholder for features
    features = []
    
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        if len(contour) >= 5:
            ellipse = cv2.fitEllipse(contour)
            (x, y), (MA, ma), angle = ellipse
            features.extend([area, perimeter, MA, ma, angle])
    
    # Ensure features array has a fixed size (e.g., pad or truncate to 20 features)
    features = features[:20]  # Take the first 20 features if more
    features += [0] * (20 - len(features))  # Pad with zeros if less

    return np.array(features)

def predict_fire_in_video(video_path):
    cap = cv2.VideoCapture(video_path)
    results = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        features = extract_features_from_frame(frame)
        features = np.append(features, 0)  # Add context feature (e.g., 0 for non-fire)

        # Convert to tensor
        features_tensor = torch.tensor(features, dtype=torch.float32)

        # Predict using the model
        with torch.no_grad():
            output = model(features_tensor)
            prediction = (output >= 0.5).float().item()
            results.append(prediction)

    cap.release()

    # Determine if the video contains fire
    fire_detected = sum(results) > (len(results) / 2)
    return fire_detected, results

# Test the model on a video
video_path = 'path_to_your_video.mp4'
fire_detected, results = predict_fire_in_video(video_path)
print(f'Fire detected: {fire_detected}')
print(f'Results for each frame: {results}')


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_save_model_weights.pth'